In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/train.parquet')
df.ts = (df.ts//1000).astype('int32')

mx = df.ts.max()
df = df.loc[df.ts >= mx-60*60*24*7]

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(52760972, 6)


,session,aid,ts,type,hour,day
147,0,10268,1661157951,0,31551,377151
148,0,219925,1661157962,0,31562,377162
149,0,1140855,1661157993,0,31593,377193
150,0,915702,1661158034,0,31634,377234
151,0,480578,1661158090,0,31690,377290


In [3]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [4]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
147,0,10268,1661157951,0,31551,377151,<NA>,-209657,0,0
148,0,219925,1661157962,0,31562,377162,209657,-920930,0,0
149,0,1140855,1661157993,0,31593,377193,920930,225153,0,0
150,0,915702,1661158034,0,31634,377234,-225153,435124,0,0
151,0,480578,1661158090,0,31690,377290,-435124,-665225,0,0


In [5]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [6]:
tmp.head()

aid
1394072     3.0
1474727    <NA>
650752     <NA>
1127302    <NA>
42004      <NA>
Name: n, dtype: float64

In [7]:
tmp.max(), tmp2.max()

(278.0, 388.0)

In [8]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

80

In [9]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
2051,69,64,6,13,-1.0,-1.0
2063,3,4,0,2,-1.0,-1.0
2070,3,4,0,1,-1.0,-1.0
2052,2,2,0,0,-1.0,-1.0
2053,3,3,0,0,-1.0,-1.0


In [10]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [11]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,11,10,43598.363636,310652.909091
1,0.000000,2,1,36721.000000,36721.000000
2,0.000000,5,4,46237.600000,322717.600000
3,0.122419,1356,648,50336.078171,268438.733038
4,0.054795,73,42,50610.931507,314545.178082


In [12]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [13]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
1813411,0.000000,20798.140625,44617.164062
1069901,-1.000000,-1.000000,-1.000000
1120135,0.000000,21527.564453,128750.429688
455508,0.957427,1724.783447,1724.783447
1484672,0.269089,18951.451172,147595.546875


In [14]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [15]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [16]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,11,10,43598.363281,310652.90625,1.100000,0.000000,22148.582031,144431.906250,9,10,0,0,-1.0,-1.0
1,0.000000,2,1,36721.000000,36721.00000,2.000000,0.000000,63.639610,63.639610,2,1,0,0,-1.0,-1.0
2,0.000000,5,4,46237.601562,322717.59375,1.250000,0.000000,9294.753906,145858.687500,2,4,0,0,-1.0,-1.0
3,0.122419,1356,648,50336.078125,268438.71875,2.092592,0.376112,16642.734375,124164.734375,152,158,23,120,1.5,1.6
4,0.054795,73,42,50610.929688,314545.18750,1.738095,0.229154,20720.726562,155550.312500,52,53,0,4,-1.0,1.5


In [17]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [18]:
COLS = [f'{x[:-1]}7' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio7', 'count_item7', 'count_user7', 'hour_mean7', 'day_mean7', 'repeat7', 'buy_ratio_std7', 'hour_std7', 'day_std7', 'prev7', 'next7', 'orders7', 'carts7', 'order_repeat7', 'cart_repeat7']


In [19]:
item_features.to_parquet('../../data/item_user_features/item20_LB.pqt')